# Math 189 - Final Report

## Names
* Nyan Aye
* Serena Xie
* Angela Shen
* Bofu Zou
* Kyounghyun Sou

#### Research Question
Do household housing cost burdens in California predict educational attainment in Californian adults? 

#### Background
Housing costs have always been a rising concern in California, especially in densely populated major cities such as Los Angeles and San Francisco. In fact, as the population grows and the housing supply is unable to keep up, California has become the third most expensive state to live in as of 2024.  As housing costs grow, households become more unlikely to spend on other matters, such as education, which will be the main focus of our study. The goal of our study is to find out if we can draw a correlation between the percentage monthly household income paid towards housing costs versus the percentage of adults in an area pursuing a 4-year college degree or higher. In broader strokes, we hope that finding said correlation would display the importance of housing issues by showing such issues can alleviate other issues, starting with education, in California. 

#### Hypothesis
There exists a correlation between the percentage of households with ‘high’ household burden (30% of household income goes towards housing) and the percentage of individuals in the area pursuing a 4-year college degree or higher within the same area.

#### Data
We found two datasets for this project from the data provided by the California government. The first one is about [California housing cost burdens](https://data.ca.gov/dataset/housing-cost-burden), with a focus on the variables percentage of income burden in household, county and region. The second dataset should be the [educational attainment dataset](https://data.ca.gov/dataset/educational-attainment), with variables county, region, and estimated percentage of educational attainment in region. Each dataset should have at least 1000 observations to be able to draw accurate estimations, with the housing cost burdens dataset utilizing a sample size of 521,264 and education attainment dataset utilizing a sample size of 166,662. We want to draw a present correlation between housing cost and education from previous historical trends, so both datasets will have a temporal coverage of 2006-2010. 

### Data Cleaning
To make our data useable, we cleaned the data based on our needs. For the educational attainment dataset, we removed the first few rows that aggregate education level by race, since this is not relevant to our goals. For the housing burden dataset, we only kept observations classified under “50% of monthly income…” and “all income levels/monthly income at all levels…income”. We also removed some columns from the table and only kept relevant data such as estimates since the dataset is too large. Afterwards, each person responsible for EDA and testing will further modify the clean data to their own needs, usually to find the mean estimates of each county for the datasets, leaving us with 58 observations.

In [3]:
import pandas as pd
import numpy as np
import re

def unicode_to_symbols(encoded_string):
    pattern = re.compile(r'_x([0-9A-Fa-f]{4})_')
    def replace_with_char(match):
        # Convert the hexadecimal value to an integer, then to the corresponding Unicode character
        return chr(int(match.group(1), 16))
    # Replace all occurrences of encoded Unicode with their corresponding characters
    return pattern.sub(replace_with_char, encoded_string)

housing_df = pd.read_excel('housing.xlsx')
education_df = pd.read_excel('education.xlsx')

In [4]:
education_df.head()
# filter down the columns into what we need
new_housing_df = housing_df.get(['ind_id', 'ind_definition','percent','burden', 'geotype', 'income_level','geotypevalue', 'geoname', 'region_name','region_code','county_fips'])

# filter to include only >50% burden
new_housing_df = new_housing_df[(new_housing_df['burden'] == '_x003E__x0020_50_x0025__x0020_of_x0020_monthly_x0020_household_x0020_income_x0020_consumed_x0020_by_x0020_monthly_x002C__x0020_selected_x002C__x0020_housing_x0020_costs')]

# filter out NA values and filter income
new_housing_df = new_housing_df.dropna()
new_housing_df = new_housing_df[(new_housing_df['income_level'] == 'All_x0020_income_x0020_levels') 
                              | (new_housing_df['income_level'] == 'Monthly_x0020_household_x0020_income_x0020_at_x0020_all_x0020_levels_x0020_of_x0020_HUD-adjusted_x0020_family_x0020_median_x0020_income')]
new_housing_df = new_housing_df.applymap(lambda x: unicode_to_symbols(x) if isinstance(x, str) else x)

# remove unnecessary columns from education.xlsx
new_education_df = education_df.get(['ind_id', 'ind_definition', 'estimate', 'geotype', 'geotypevalue', 'geoname','region_name', 'region_code', 'county_fips']).dropna()

new_housing_df.to_excel('clean_housing.xlsx')
new_education_df.to_excel('clean_education.xlsx')

C:\Users\nyanl\AppData\Local\Temp\ipykernel_30812\2626337546.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  new_housing_df = new_housing_df.applymap(lambda x: unicode_to_symbols(x) if isinstance(x, str) else x)


In [8]:
new_housing_df.head(5)

,ind_id,ind_definition,percent,burden,geotype,income_level,geotypevalue,geoname,region_name,region_code,county_fips
813,106,Percent of households spending more than 30% (...,17.217629,> 50% of monthly household income consumed by ...,CO,Monthly household income at all levels of HUD-...,06001,Alameda,Bay Area,01,06001
823,106,Percent of households spending more than 30% (...,22.556391,> 50% of monthly household income consumed by ...,CO,All income levels,06001,Alameda,Bay Area,01,06001
829,106,Percent of households spending more than 30% (...,17.071468,> 50% of monthly household income consumed by ...,CO,All income levels,06001,Alameda,Bay Area,01,06001
835,106,Percent of households spending more than 30% (...,28.111467,> 50% of monthly household income consumed by ...,CO,All income levels,06001,Alameda,Bay Area,01,06001
841,106,Percent of households spending more than 30% (...,23.745072,> 50% of monthly household income consumed by ...,CO,All income levels,06001,Alameda,Bay Area,01,06001


In [11]:
new_education_df.head(5)

,ind_id,ind_definition,estimate,geotype,geotypevalue,geoname,region_name,region_code,county_fips
9,355,Percent of population age 25 and up with a fou...,21.1,CD,607192260,Ontario CCD,Southern California,14.0,6071.0
10,355,Percent of population age 25 and up with a fou...,0.1,CD,603593280,Susanville CCD,Northeast Sierra,6.0,6035.0
11,355,Percent of population age 25 and up with a fou...,17.0,CD,607192100,Newberry-Baker CCD,Southern California,14.0,6071.0
12,355,Percent of population age 25 and up with a fou...,19.9,CD,609792940,Santa Rosa CCD,Bay Area,1.0,6097.0
13,355,Percent of population age 25 and up with a fou...,28.3,CD,600190020,Alameda CCD,Bay Area,1.0,6001.0


#### Exploratory Data Analysis
* Side-by-side choropleth map of housing burden and education attainment will show how to 
distribution of the two datasets look in California when viewed side by side
* We’ll also use a heat map to communicate relationships between the variables.
* Ordinary Least Squares Regression will be used to conduct a linear regression to find the 
relation between average percentage of high housing burden and average secondary
education attainment per county. Will also include a scatterplot as visualization to show 
what the correlation looks like. s like.

#### Hypothesis Test